In [ ]:
# default_exp core

# 01 Prodb Benchmarks

> Which database is faster? The highly optimised SQLAlchemy library, or the library that literally saves and loads the entire csv every time you modify it?

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
import pandas as pd
import arrow
import os
from time import gmtime, strftime
import sys; sys.path.append('../')
from prodb.core import generate_db, insert_row, insert_rows, utc_now, readable_df

# 1. Prodb API Benchmarks

In [ ]:
# ================================================================ #
data = {'name': ['Sam', 'Grant'],
        'mood': ['😊', '😵'],
        'message': ['hello from London, UK', 'hello from Christchurch, NZ'],
        'time_utc' : [utc_now(), utc_now()]}
# ================================================================ #

Duration to insert, append, save, and reload the dataframe 100 times.

In [ ]:
%%time

def test_prodb(n=100):
    dbpath='benchmarks/prodb.csv'
    df = generate_db(dbpath=dbpath, cols='name mood message'.split())
    for i in range(n):
        df = insert_rows(df, data, dbpath=dbpath)   # save to disk
        df = pd.read_csv(dbpath)                    # load from disk
    return df

df = test_prodb()
print(df.shape)
display(df.tail())

✓💾 benchmarks/prodb.csv (19 kb)
(200, 4)


,name,mood,message,time_utc
195,Grant,😵,"hello from Christchurch, NZ",2021-11-06 22:12:35
196,Sam,😊,"hello from London, UK",2021-11-06 22:12:35
197,Grant,😵,"hello from Christchurch, NZ",2021-11-06 22:12:35
198,Sam,😊,"hello from London, UK",2021-11-06 22:12:35
199,Grant,😵,"hello from Christchurch, NZ",2021-11-06 22:12:35


Wall time: 456 ms


#### Reading and writing time of small `csv`

In [ ]:
df.shape

(200, 4)

In [ ]:
%%timeit
df.to_csv('benchmarks/tmp.csv')

1.09 ms ± 22.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
df = pd.read_csv('benchmarks/tmp.csv')

1.21 ms ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%%timeit
df.iloc[50, :]

67.3 µs ± 2.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# 2.  SQLModel / SQLAlchemy Benchmarks
`SQLModel` code and functions from [lukexyz/sqlmodel-streamlit/app.py](https://github.com/lukexyz/sqlmodel-streamlit/blob/main/app.py)

In [ ]:
from sqlmodel import Field, Session, SQLModel, create_engine, select
from typing import Optional

import logging, sys
logging.disable(sys.maxsize)


class Hero(SQLModel, table=True):
    __table_args__ = {'extend_existing': True}  # required for streamlit refreshing
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

def create_db_and_tables():
    SQLModel.metadata.create_all(engine)

def get_db_size():
    with Session(engine) as session:
        heroes = session.exec(select(Hero)).all()
    return len(heroes)

def select_heros():
    with Session(engine) as session:
        statement = select(Hero).where(Hero.age <= 35)
        results = session.exec(statement)
        for hero in results:
            print(hero)

def show_table():
    with Session(engine) as session:
        heroes = session.exec(select(Hero)).all()
        display(pd.DataFrame([s.dict() for s in heroes[-5:]]))

def get_table():
    with Session(engine) as session:
        heroes = session.exec(select(Hero)).all()
        return pd.DataFrame([s.dict() for s in heroes])

def delete_db():
    with Session(engine) as session:
        heroes = session.exec(select(Hero)).all()
        for hero in heroes:
            session.delete(hero)  
        session.commit()  

def commit_new_row():
    hero = Hero(name='Luke', secret_name='Luke Woods', age=23)
    hero_1 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=36)
    with Session(engine) as session:
        session.add(hero)
        session.add(hero_1)
        session.commit()

D:\ProgramData\Anaconda3\envs\prodb\lib\site-packages\sqlmodel\main.py:367: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Hero, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_used, **kw)


> Create engine

In [ ]:
sqlite_file_name = 'sqlmodel.db'
sqlite_url = f"sqlite:///{sqlite_file_name}"
engine = create_engine(sqlite_url, echo=True)

In [ ]:
delete_db()
create_db_and_tables()
commit_heroes()
show_table()
print(f'length {get_db_size()}')

,age,id,name,secret_name
0,36,1,Rusty-Man,Tommy Sharp
1,33,2,Dr. Weird,Steve Weird


length 2


### Run 100 iteration benchmark

In [ ]:
%%time
delete_db()
create_db_and_tables()
for k in range(100):
    commit_new_row()
    #write_new_row()  # add two rows to db
df = get_table()
print(df.shape)
df.tail()

(200, 4)
Wall time: 13.6 s


,age,id,name,secret_name
195,36,196,Rusty-Man,Tommy Sharp
196,23,197,Luke,Luke Woods
197,36,198,Rusty-Man,Tommy Sharp
198,23,199,Luke,Luke Woods
199,36,200,Rusty-Man,Tommy Sharp


# SQLAlchemy
* Good code from the good man, Jcharis on [github](https://github.com/Jcharis/streamlit_todo_crud_app/blob/main/db_fxns.py)

In [ ]:
#!pip install sqlalchemy

In [ ]:
import pandas as pd
import json
import sqlite3

In [ ]:
conn = sqlite3.connect('benchmarks/sqlite.db',check_same_thread=False)
c = conn.cursor()

def create_table():
    c.execute('CREATE TABLE IF NOT EXISTS taskstable(name TEXT,mood TEXT,message TEXT)')

def add_data(name,mood,message):
    c.execute('INSERT INTO taskstable(name,mood,message) VALUES (?,?,?)',(name,mood,message))
    conn.commit()
    
def view_all_data():
    c.execute('SELECT * FROM taskstable')
    data = c.fetchall()
    return data

def empty_table(name):
    c.execute('DELETE FROM taskstable WHERE name="{}"'.format(name))
    conn.commit()

In [ ]:
create_table()

In [ ]:
name = "Luke"
mood = "👍"
message = "An SQLite database entry"
empty_table(name)
add_data(name, mood, message)

In [ ]:
ds = pd.DataFrame(view_all_data(), columns=['name', 'mood', 'message'])
print(ds.shape)
ds.tail()

(1, 3)


,name,mood,message
0,Luke,👍,An SQLite database entry


### `SQLAlchemy` 100 commits benchmark

In [ ]:
%%time
for i in range(100):
    add_data(name, mood, message)
res = view_all_data()

Wall time: 10.9 s
